In [1]:
import scipy.io
import numpy as np
import pandas as pd
from skspatial.objects import Line
from skspatial.objects import Points

In [2]:
directory = r'C:\Users\shirahad\Documents\research\electrophysiology\analyze_electrophysiology_data\mapping\files'

In [3]:
probe_coordinates = scipy.io.loadmat(os.path.join(directory, 'probe_points_electrode_track.mat'))
channel_positions = np.load(os.path.join(directory, 'channel_positions.npy'))
channel_map = np.load(os.path.join(directory, 'channel_map.npy'))
clusters_data_df = pd.read_csv(os.path.join(directory, 'cluster_info.tsv'), sep="\t", header=0, index_col=0)

In [4]:
channel_positions

array([[ 43.3, 775. ],
       [ 43.3, 675. ],
       [ 43.3, 475. ],
       [ 43.3, 375. ],
       [ 43.3, 325. ],
       [ 43.3, 125. ],
       [ 43.3, 725. ],
       [ 43.3,  25. ],
       [  0. ,  50. ],
       [  0. , 300. ],
       [  0. , 150. ],
       [  0. , 700. ],
       [  0. , 500. ],
       [  0. , 350. ],
       [  0. , 650. ],
       [  0. , 200. ],
       [ 43.3, 225. ],
       [ 43.3, 275. ],
       [ 43.3, 575. ],
       [ 43.3, 525. ],
       [ 43.3, 425. ],
       [ 43.3, 175. ],
       [ 43.3, 625. ],
       [ 43.3,  75. ],
       [  0. ,   0. ],
       [  0. , 450. ],
       [  0. , 100. ],
       [  0. , 250. ],
       [  0. , 600. ],
       [  0. , 400. ],
       [  0. , 550. ],
       [  0. , 750. ]])

In [5]:
probe_number = 1 # number in registration
probe_type = 'neuronexus_A1_32_Poly2' # cambridge_neurotech_H7b

In [6]:
borders_table = pd.read_csv(os.path.join(directory, 'borders_table_{}.csv'.format(probe_number)), header=0, index_col=None)
probe_data = pd.read_csv(os.path.join(directory, 'probe_data_{}.csv'.format(probe_number)), header=0, index_col=None)

In [7]:
borders_table

,upperBorder,lowerBorder,acronym,name,avIndex
0,0,200,MOs1,Secondary motor area layer 1,26
1,200,280,MOs2/3,Secondary motor area layer 2/3,27
2,280,850,ACAd2/3,Anterior cingulate area dorsal part layer 2/3,229
3,850,1810,PL2/3,Prelimbic area layer 2/3,242
4,1810,1840,ILA2/3,Infralimbic area layer 2/3,249
5,1840,2090,ILA5,Infralimbic area layer 5,250
6,2090,2180,ILA5,Infralimbic area layer 5,250
7,2180,2220,ACAv5,Anterior cingulate area ventral part layer 5,236
8,2220,2230,ILA5,Infralimbic area layer 5,250
9,2230,2240,ACAv5,Anterior cingulate area ventral part layer 5,236


In [34]:
probe_coordinates['pointList'][0][0][0][probe_number - 1][0]

array([[542, 431, 402],
       [545, 396, 402],
       [541, 359, 402],
       [542, 325, 402],
       [542, 285, 402],
       [548, 385, 387],
       [547, 358, 387],
       [547, 336, 387],
       [546, 302, 387],
       [547, 274, 387],
       [547, 258, 387],
       [544, 356, 375],
       [547, 337, 375],
       [547, 314, 375],
       [547, 290, 375],
       [547, 259, 375],
       [547, 239, 375],
       [542, 323, 360],
       [544, 299, 360],
       [542, 276, 360],
       [542, 249, 360],
       [541, 225, 360],
       [555, 301, 348],
       [551, 275, 348],
       [545, 246, 348],
       [543, 223, 348],
       [541, 194, 348],
       [547, 329, 339],
       [545, 303, 339],
       [542, 275, 339],
       [539, 247, 339],
       [537, 224, 339],
       [537, 197, 339],
       [547, 367, 330],
       [545, 336, 330],
       [543, 305, 330],
       [541, 276, 330],
       [537, 246, 330],
       [532, 216, 330],
       [531, 185, 330]], dtype=uint16)

In [9]:
def get_probe(probe_number, probe_coordinates):
    probe_df = pd.DataFrame(probe_coordinates['pointList'][0][0][0][probe_number - 1][0], columns=['ML', 'DV', 'AP'])
    probe_df = probe_df * 10 # convert pixels to micorns
    return probe_df

In [29]:
probe_df = get_probe(probe_number, probe_coordinates)

In [ ]:
def get_probe_best_fit_line(probe_df):
    points = Points(probe_df.to_numpy())
    line_fit = Line.best_fit(points)
    return line_fit

In [ ]:
probe_best_fit_line = get_probe_best_fit_line(probe_df[['ML', 'DV', 'AP']])

In [12]:
def is_value_in_range(range_df, value):
    return range_df['upperBorder'] <= value < range_df['lowerBorder']

In [31]:
def add_acronym(df, borders_table):
    for i, row in df.iterrows():
        region = borders_table[borders_table[['lowerBorder', 'upperBorder']].apply(is_value_in_range, value=row['DV'], axis=1)]
        df.at[i, 'region_name'] = region['name'].item() if not region.empty else ''
        df.at[i, 'region_acronym'] = region['acronym'].item() if not region.empty else ''
    return df

In [32]:
probe_df = add_acronym(probe_df, borders_table)

In [33]:
probe_df

,ML,DV,AP,region_name,region_acronym
0,4753.587766,3780.067025,3525.723768,,
1,4779.899138,3473.101025,3525.723768,,
2,4744.817309,3148.594111,3525.723768,Striatum,STR
3,4753.587766,2850.398568,3525.723768,Taenia tecta dorsal part,TTd
4,4753.587766,2499.580283,3525.723768,Dorsal peduncular area,DP
5,4806.210509,3376.625996,3394.166911,,
6,4797.440052,3139.823654,3394.166911,Striatum,STR
7,4797.440052,2946.873597,3394.166911,Taenia tecta dorsal part,TTd
8,4788.669595,2648.678054,3394.166911,Taenia tecta dorsal part,TTd
9,4797.440052,2403.105255,3394.166911,Dorsal peduncular area,DP


In [36]:
def get_good_clusters_coordinates(clusters_df, channel_positions, channel_map):
    good_clusters_df = clusters_df.loc[(clusters_df['KSLabel'] == 'good') | (clusters_df['group'] == 'good'), ['ch', 'n_spikes']]
    probe_mapping_df = pd.DataFrame({'coordinates': list(map(tuple, channel_positions))}, index = channel_map.flatten())
    return good_clusters_df.join(probe_mapping_df, on='ch', how='left')

In [37]:
clusters_df = get_good_clusters_coordinates(clusters_data_df, channel_positions, channel_map)

In [38]:
clusters_df

,ch,n_spikes,coordinates
id,,,
3,15,73563,"(0.0, 200.0)"
8,30,247,"(0.0, 550.0)"
34,5,6664,"(43.3, 125.0)"
68,2,64061,"(43.3, 475.0)"
70,2,2091,"(43.3, 475.0)"


In [ ]:
config = yaml.safe_load(open(r'C:\Users\shirahad\Documents\research\electrophysiology\analyze_electrophysiology_data\mapping\configuration\{}_config.yml'.format(probe_type))) # fix path

In [ ]:
config

In [ ]:
#active_probe_length = config['distance_between_channels_same_column'] * (config['num_of_channel_per_column'] - 1) + config['shift_between_columns']

In [ ]:
def get_point_coordinates_on_probe(probe_points, cluster, shrinkage_factor):
    point = probe_points.to_point(cluster['coordinates'][1] * shrinkage_factor) # get the relevant point on the probe
    point[0] = point[0] + cluster['coordinates'][0] # shift the point on the ML axis (according to the columns on the probe)
    return point

In [ ]:
clusters_points = clusters_df.apply(lambda x: get_point_coordinates_on_probe(probe_best_fit_line, x, probe_data['shrinkage_factor'].item()), axis=1)
clusters_points_df = pd.DataFrame(clusters_points.values.tolist(), index=clusters_points.index, columns=['ML', 'DV', 'AP'])
clusters_df = clusters_df.join(clusters_points_df, how='left')

In [ ]:
clusters_df = add_acronym(clusters_df, borders_table)

In [ ]:
clusters_df

In [ ]:
# calculate probes registration points locations (from SHARP-TRACK) )in microns and rename columns to match brain_render demands
probe_df.columns = ['x', 'y', 'z', 'region_name', 'region_acronym']
fitted_probe_df = pd.DataFrame({'point': probe_best_fit_line.point, 'direction': probe_best_fit_line.direction})

In [ ]:
clusters_coordinates_df = clusters_df[['ML', 'DV', 'AP', 'region_name', 'region_acronym']].copy()

In [ ]:
clusters_coordinates_df.columns = ['x', 'y', 'z', 'region_name', 'region_acronym']

In [ ]:
fitted_probe_df.to_csv(r'{}\files\probe_fitted_line_{}.csv'.format(directory, probe_number), sep=',', encoding='utf-8')
probe_df.to_csv(r'{}\files\sharp_track_probe_coordinates_{}.csv'.format(directory, probe_number), sep=',', encoding='utf-8', index_label='id')